In [2]:
#데모
#위키 문서로부터 학습하는 텐서플로우의 LSTM RNN 을 사용해 
#새로운 데이터 세트 https://metamind.io/research/the-wikitext-long-term-dependency-language-modeling-dataset/ 를 생성합니다
#위키피디아 기사로부터 1억 개의 표시를 불러오고 , MetaMind bought by Salesforce
#big companies are buying AI startups like candy
#우리는 케라스 대신 텐서플로우의 TF 를 사용하고
#LSTM 의 내장함수를 사용합니다

#테스트 입력 문자열과 500 개의 문자를 출력합니다

import numpy as np #벡터화
import random #확률분포를 생성합니다 
import tensorflow as tf #ml
import datetime #클록 훈련 시간

In [3]:
#텍스트를 열어봅시다
#파이썬 내장 파일 읽기 함수
text = open('wiki.test.raw').read()
print('text length in number of characters:', len(text))

print('head of text:')
print(text[:1000]) #표식이 붙은 단어로 text 라는 리스트에 저장되어 있습니다

('text length in number of characters:', 1290590)
head of text:
 
 = Robert Boulter = 
 
 Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy 's Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . 
 In 2006 , Boulter starred alongside Whishaw in the 

In [4]:
#세트는 반복되는 요소가 없는 정렬되지 않은 것입니다.
#리스트로 변환한 뒤, 모든 문자에 대해 
#알파벳 순서로 정렬합니다
chars = sorted(list(set(text)))
char_size = len(chars)
print('number of characters:', char_size)
print(chars)
print('hello')

('number of characters:', 178)
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x81', '\x82', '\x83', '\x84', '\x85', '\x86', '\x87', '\x88', '\x89', '\x8a', '\x8b', '\x8c', '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9b', '\x9c', '\x9d', '\x9e', '\x9f', '\xa0', '\xa1', '\xa2', '\xa3', '\xa4', '\xa5', '\xa6', '\xa7', '\xa8', '\xa9', '\xaa', '\xab', '\xad', '\xae', '\xaf', '\xb0', '\xb1', '\xb2', '\xb3', '\xb4', '\xb5', '\xb6', '\xb7', '\xb8', '\xb9', '\xba', '\xbb', '\xbc', '\xbd', '\xbe', '\xbf', '\xc2', '\xc3', '\xc4', '\

In [5]:
#문자를 id 로, id 를 문자로 변환하는
#문자를 숫자로 숫자를 문자로 매핑하는 딕셔너리
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))
#print(id2char)

In [6]:
#각 문자의 확률을 가지고 원-핫 인코딩된 가장 확률이 높은 문자를 반환합니다
#예측값으로 각 문자의 확률을 가진 배열을 알 수 있고
#그 중에서 가장 확률이 높은 것을 원-핫 인코딩합니다
def sample(prediction):
    #샘플은 반열린구간 내에 균일 분포되어있습니다
    r = random.uniform(0,1)
    #예상 문자를 저장합니다
    s = 0
    #길이 > 지수 이기 때문에 0 부터 시작합니다
    char_id = len(prediction) - 1
    #각 문자별 예상 확률
    for i in range(len(prediction)):
        #S 에 할당합니다
        s += prediction[i]
        #확률이 무작위로 생성된 것보다 큰지 확인합니다
        if s >= r:
            #만약 그렇다면 그것이 다음 문자입니다
            char_id = i
            break
    #dont try to rank, just differentiate 
    #initialize the vector 벡터를 초기화합니다
    char_one_hot = np.zeros(shape=[char_size])
    #인코딩된 문자 ID
    #https://image.slidesharecdn.com/latin-150313140222-conversion-gate01/95/representation-learning-of-vectors-of-words-and-phrases-5-638.jpg?cb=1426255492
    char_one_hot[char_id] = 1.0
    return char_one_hot

In [8]:
#모델에 맞게 데이터를 벡터화합니다

len_per_section = 50
skip = 2
sections = []
next_chars = []
#리스트를 문자로 채웁니다.
#문자 단계에서 실행하는 것이기 때문에
#2 개의 문자마다 50 개의 문자를 생성합니다
for i in range(0, len(text) - len_per_section, skip):
    sections.append(text[i: i + len_per_section])
    next_chars.append(text[i + len_per_section])
#입력과 출력을 벡터화합니다
#섹션의 길이와 문자의 수로 구성된 행렬
X = np.zeros((len(sections), len_per_section, char_size))
# 문자 ID 가 여전히 0 인 열을 찾아냅니다
y = np.zeros((len(sections), char_size))
#섹션의 각 문자마다 ID 로 변환합니다
#각 섹션마다 라벨을 ID 로 변환합니다
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1
print(y)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [7]:
#배치의 크기가 신경망을 거칠 샘플의 수를 결정합니다
#1 epoch = 모든 훈련 샘플의 한 번의 순방향과 역방향 과정
#배치의 크기 = 하나의 순방향과 역방향 과정에서의 훈련 샘플의 수
#배치의 크기가 클 수록 필요한 메모리 공간이 많습니다
#1000 개의 훈련 샘플이 있고
#500 의 배치 크기를 가진다면, 1 epoch 을 마치려면 2 번의 반복이 필요합니다
batch_size = 512
#총 반복 수
max_steps = 72001
#로그 기록 주기
log_every = 100
#저장 주기
save_every = 6000
#은닉층에 너무 적은 수의 뉴런이 있다면 과소적합이 발생해
#데이터 세트의 신호를 적절히 감지하지 못할 수 있습니다
#너무 많다면 과대적합이 발생합니다
hidden_nodes = 1024
#시작문
test_start = 'I am thinking that'
#모델을 저장합니다
checkpoint_directory = 'ckpt'

#체크포인트 디렉토리를 생성합니다
if tf.gfile.Exists(checkpoint_directory):
    tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('training data size:', len(X))
print('approximate steps per epoch:', int(len(X)/batch_size))


hello
('training data size:', 645270)
('approximate steps per epoch:', 1260)


In [8]:
#모델 시간을 구축합니다
#계산 그래프를 생성합니다
graph = tf.Graph()
#다수의 그래프가 있으면 사용되지만, 여기서는 하나만 있습니다
with graph.as_default():
    ###########
    #준비
    ###########
    #변수와 플레이스홀더
    #global_step 은 그래프에 나타난 배치의 수를 의미합니다
    #배치가 제공될 때마다 가중치는
    #손실을 최소화하는 방향으로 갱신됩니다.
    #global_step 은 0 부터 시작해 발견된 배치 수를 따라갑니다.
    global_step = tf.Variable(0)
    
    #텐서를 섹션에 적용합니다
    data = tf.placeholder(tf.float32, [batch_size, len_per_section, char_size])
    #라벨
    labels = tf.placeholder(tf.float32, [batch_size, char_size])
    
    #LSTM RNN (Long Short Term Memory) 는 3 개의 게이트와 내부 상태로 이루어져 있습니다. 
    #이것으로 LSTM 이 장기 종속성을 발견할 수 있습니다.
    #http://suriyadeepan.github.io/2017-02-13-unfolding-rnn-2/
    #셀 상태와 3 개의 게이트에 대한 평중과 가중치를 생성해봅시다.
    
    #tf Variable
    #모델에 대한 가중치와 편중이 필요하기 때문에
    #이것을 마치 추가적인 입력으로 처리한다고 볼 수도 있습니다.
    #하지만 텐서플로우에서는 Variable을 사용하는 것이 더 나을 수 있습니다.
    #Variable 은 텐서플로우의 상호 연산 그래프에 있는 수정 가능한 텐서입니다
    #계산에 사용될 수도 있고, 계산에 의해 수정될 수도 있습니다.
    #머신러닝 응용프로그램에서는 매개변수를 Variable 로 사용합니다.
    
    #예비 LSTM 연산
    #입력 게이트: 입력에 대한 가중치, 이전 출력에 대한 가중치, 편중
    
    #tf truncated normal
    #절단 정규분포로부터 무작위값을 출력합니다.
    #생성된 값은 지정된 정규분포의 평균과 표준편차를 가지지만
    #평균으로부터 2 표준편차 이상 떨어져있는 값들은
    #탈락시킵니다.
    #무작위로 초기화된 값들입니다.
    
    #b편중값이 닻과 같은 역할을 합니다

    w_ii = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_io = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_i = tf.Variable(tf.zeros([1, hidden_nodes]))
    #망각 게이트: 입력에 대한 가중치, 이전 출력값에 대한 가중치, 편중
    w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_fo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_f = tf.Variable(tf.zeros([1, hidden_nodes]))
    #출력 게이트: 입력에 대한 가중치, 이전 출력값에 대한 가중치, 편중
    w_oi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_oo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_o = tf.Variable(tf.zeros([1, hidden_nodes]))
    #기억 셀: 입력에 대한 가중치, 이전 출력값에 대한 가중치, 편중
    w_ci = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_co = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_c = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    
    #LSTM 셀
    #입력값, 출력값과 외부 상태를 받아 출력과 상태를 반환합니다
    #출력은 빈 상태로 시작하고 LSTM 셀이 계산하게 됩니다.
    
    #내부와 외부 두 개의 상태가 있고
    #두 가지가 모두 있어야만
    #이후의 연산을 이어나갈 수 있기 때문에
    #각 과정에서 그들을 텐서로 결합하고 다음 과정의 입력으로 전달합니다
    #이 텐서들을 각 과정의 처음 부분에 st_1 과 ct_1 로 풀어집니다
    
    
    def lstm(i, o, state):
        
        #이것들은 개별적으로 계산되어서 서로 중복되지 않습니다
        #(입력값 * 입력값의 가중치) + (출력값 * 이전 출력값의 가중치) + 편중
        input_gate = tf.sigmoid(tf.matmul(i, w_ii) + tf.matmul(o, w_io) + b_i)
        #(입력값 * 망각 가중치) + (출력값 * 이전 출력값의 가중치) + 편중
        forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo) + b_f)
        #(입력값 * 출력값의 가중치) + (출력값 * 이전 출력값의 가중치) + 편중
        output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
        #(입력값 * 내부 상태의 가중치) + (출력값 * 이전 출력값의 가중치) + 편중
        memory_cell = tf.sigmoid(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)
        
        #망각 게이트 * 주어진 상태 +  입력 게이트 * 은닉 상태
        state = forget_gate * state + input_gate * memory_cell
        #비선형 tanh 로 상태값을 압축합니다 (x 원소에 대해 쌍곡탄젠트값을 계산합니다)
        #출력값을 곱해줍니다
        output = output_gate * tf.tanh(state)
        #반환합니다
        return output, state
    
    ###########
    #연산
    ###########
    #LSTM
    #빈 상태로 시작하고 LSTM 이 계산을 하게 됩니다
    output = tf.zeros([batch_size, hidden_nodes])
    state = tf.zeros([batch_size, hidden_nodes])

    #LSTM 루프를 풀어줍니다
    #각각의 입력 세트마다 실행해줍니다
    for i in range(len_per_section):
        #LSTM 으로부터 상태와 출력을 계산합니다
        output, state = lstm(data[:, i, :], output, state)
        #시작
        if i == 0:
            #초기 출력과 라벨을 저장합니다
            outputs_all_i = output
            labels_all_i = data[:, i+1, :]
        #각각의 새로운 세트마다 출력과 라벨을 이어줍니다
        elif i != len_per_section - 1:
            #차원축 위의 벡터를 이어서 결합해줍니다
            outputs_all_i = tf.concat(0, [outputs_all_i, output])
            labels_all_i = tf.concat(0, [labels_all_i, data[:, i+1, :]])
        else:
            #최종 저장
            outputs_all_i = tf.concat(0, [outputs_all_i, output])
            labels_all_i = tf.concat(0, [labels_all_i, labels])
        
    #Classifier
    #The Classifier 는 saved_output 과 saved_state 이 지정되고 난 후에만 실행됩니다.
    
    #무작위의 크기와 분포로 생성된 신경망의
    #가중치와 편중을 계산합니다.
    w = tf.Variable(tf.truncated_normal([hidden_nodes, char_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([char_size]))
    #로지트는 단순히 함수가 이전 층의 출력값에 실행된다는 것을 의미하며
    #단위를 이해하기 위한 상대적 비율은 선형입니다.
    #그 말인 즉슨, 입력값이 확률이 아니기 때문에
    #합이 1 이 아닐 수 있습니다. (5 의 입력값을 가집니다)
    logits = tf.matmul(outputs_all_i, w) + b
    
    #로지트는 예상 출력입니다.
    #다중클래스 분급기 소프트맥스 층의 비용과
    #텐서의 차원 내 요소들의 평균을 계산하기 때문에
    #이것을 교차 엔트로피와 비교해봐야 합니다.
    #모든 값의 평균 손실
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels_all_i))

    #최적화
    #경사 하강법을 이용한 손실을 최소화하고, 10 의 학습 속도를 가지며, 배치를 기억합니다.
    optimizer = tf.train.GradientDescentOptimizer(10.).minimize(loss, global_step=global_step)
    
    ###########
    #테스트
    ###########
    #test_data = tf.placeholder(tf.float32, shape=[1, char_size])
    #test_output = tf.Variable(tf.zeros([1, hidden_nodes]))
    #test_state = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    #각 테스트의 시작 단계마다 재설정해줍니다
    #reset_test_state = tf.group(test_output.assign(tf.zeros([1, hidden_nodes])), 
                                #test_state.assign(tf.zeros([1, hidden_nodes])))

    #LSTM
    #test_output, test_state = lstm(test_data, test_output, test_state)
    #test_prediction = tf.nn.softmax(tf.matmul(test_output, w) + b)

In [9]:
#그래프로 세션을 초기화합니다
with tf.Session(graph=graph) as sess:
    #표준 초기화 과정
    tf.global_variables_initializer().run()
    offset = 0
    saver = tf.train.Saver()
    
    #매 훈련 과정마다 반복해줍니다
    for step in range(max_steps):
        
        #0 으로 시작합니다
        offset = offset % len(X)
        
        #배치 데이터와 라벨을 계산해 모델에 적용합니다
        if offset <= (len(X) - batch_size):
            #첫 단계
            batch_data = X[offset: offset + batch_size]
            batch_labels = y[offset: offset + batch_size]
            offset += batch_size
        #offset = batch_size 일 때까지 실행하고, 그 다음은
        else:
            #마지막 단계
            to_add = batch_size - (len(X) - offset)
            batch_data = np.concatenate((X[offset: len(X)], X[0: to_add]))
            batch_labels = np.concatenate((y[offset: len(X)], y[0: to_add]))
            offset = to_add
        
        #최적화!!
        _, training_loss = sess.run([optimizer, loss], feed_dict={data: batch_data, labels: batch_labels})
        
        if step % 10 == 0:
            print('training loss at step %d: %.2f (%s)' % (step, training_loss, datetime.datetime.now()))

            if step % save_every == 0:
                saver.save(sess, checkpoint_directory + '/model', global_step=step)

training loss at step 0: 5.15 (2017-03-21 15:46:13.367470)
I am thinking that        e                                                                                                                                  e                                                                                                                                                                                                                                                                                                                                                                        
training loss at step 10: 6.73 (2017-03-21 15:48:13.244305)
training loss at step 20: 3.22 (2017-03-21 15:50:01.814675)


KeyboardInterrupt: 

In [ ]:
test_start = 'I plan to make the world a better place '

with tf.Session(graph=graph) as sess:
    #그래프를 초기화하고, 모델을 불러옵니다
    tf.global_variables_initializer().run()
    model = tf.train.latest_checkpoint(checkpoint_directory)
    saver = tf.train.Saver()
    saver.restore(sess, model)

    #입력 변수를 설정해 문자를 생성합니다
    reset_test_state.run() 
    test_generated = test_start

    #입력 문장의 모든 문자마다 반복
    for i in range(len(test_start) - 1):
        #빈 문자 저장공간을 초기화합니다
        test_X = np.zeros((1, char_size))
        #아래로부터 가져와 id 로 저장합니다
        test_X[0, char2id[test_start[i]]] = 1.
        #모델에 적용하면, test_prediction 이 출력값이 됩니다
        _ = sess.run(test_prediction, feed_dict={test_data: test_X})

    
    #인코딩된 예상 문자를 저장하는 곳입니다
    test_X = np.zeros((1, char_size))
    test_X[0, char2id[test_start[-1]]] = 1.

    #500 개의 문자를 생성해봅시다
    for i in range(500):
        #각 예상값의 확률을 구합니다
        prediction = test_prediction.eval({test_data: test_X})[0]
        #원-핫 인코딩을 실행합니다
        next_char_one_hot = sample(prediction)
        #최대값(최고 확률값)를 찾고 문자로 변환합니다
        next_char = id2char[np.argmax(next_char_one_hot)]
        #각 문자를 출력 글에 추가해줍니다
        test_generated += next_char
        #갱신해줍니다
        test_X = next_char_one_hot.reshape((1, char_size))

    print(test_generated)